# QBOB |👷🏾‍♂️> examples

## Generate a `HelloWorld` Q# operation

In [1]:
from qbob import qbob, qpam
from qbob.intrinsics import Message, M, H

In [2]:
my_qbob = qbob.OperationBuilder("HelloWorld", entrypoint=True)
my_qbob += Message("Hello World!")
with my_qbob.allocate_qubit("q") as q:
    my_qbob += H(q)
    my_qbob.returns(M(q))

In [3]:
print(my_qbob.build())

@EntryPoint()
operation HelloWorld () : Result {
    Message("Hello World!");
    using (q = Qubit()) {
        H(q);
        return M(q);
    }
}


In [4]:
my_qbob = qbob.OperationBuilder("HelloWorld", entrypoint=True, debug=True)
my_qbob += Message("Hello World!")
with my_qbob.allocate_qubits("q", 1) as q:
    my_qbob += H(q)
    my_qbob.log_state(q)
    my_qbob.returns(M(q))

In [5]:
print(my_qbob.build())

Preparing Q# environment...
Hello World!
@EntryPoint()
operation HelloWorld () : Result {
    Message("Hello World!");
    using (q = Qubit()) {
        H(q);
        // # wave function for qubits with ids (least to most significant): 0
        // ∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
        // ∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
        return M(q);
    }
}


In [6]:
hello_world = my_qbob.compile()

In [7]:
hello_world()

Hello World!


1

## QPAM |👷🏻‍♀️>: Generate a Q# project

In [8]:
my_qpam = qpam.ProgramArchitect("MyProject")
my_qpam.add_operations(my_qbob)
project_file = my_qpam.create_project(".")

Hello World!


In [9]:
!cat $project_file

<Project Sdk="Microsoft.Quantum.Sdk/0.14.2011120240">
    <PropertyGroup>
        <OutputType>Exe</OutputType>
        <TargetFramework>netcoreapp3.1</TargetFramework>
    </PropertyGroup>
</Project>

In [10]:
!cat Program.qs

namespace MyProject
        {
            open Microsoft.Quantum.Canon;
            open Microsoft.Quantum.Intrinsic;
            @EntryPoint()
operation HelloWorld () : Result {
    Message("Hello World!");
    using (q = Qubit()) {
        H(q);
        // # wave function for qubits with ids (least to most significant): 0
        // ∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
        // ∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
        return M(q);
    }
}
        }

In [11]:
!dotnet run --project $project_file

========Hello World!
Zero
=

## Teleport

In [12]:
from qbob.intrinsics import H, CNOT, X, Z, Measure
from qbob.types import Qubit, Pauli, One

teleport = qbob.OperationBuilder("Teleport")
msg = teleport.input("msg", Qubit)
target = teleport.input("target", Qubit)

with teleport.allocate_qubits("auxiliary", 1) as aux:
    teleport += H(aux)
    teleport += CNOT(aux, target)
    teleport += CNOT(msg, aux)
    teleport += H(msg)

    with teleport.if_statement(Measure([Pauli.PauliZ], [msg]) == One):
        teleport += Z(target)

    with teleport.if_statement(Measure([Pauli.PauliZ], [aux]) == One):
        teleport += X(target)

print(teleport.build())

operation Teleport (msg : Qubit, target : Qubit) : Unit {
    using (auxiliary = Qubit()) {
        H(auxiliary);
        CNOT(auxiliary, target);
        CNOT(msg, auxiliary);
        H(msg);

        if (Measure([PauliZ], [msg]) == One) {
            Z(target);
        }

        if (Measure([PauliZ], [auxiliary]) == One) {
            X(target);
        }
    }
}


In [13]:
teleport.compile()

<Q# callable Teleport>

In [14]:
my_qbob = qbob.OperationBuilder("TeleportMessage")
message = my_qbob.input("message", bool)

with my_qbob.allocate_qubits("register", 2) as register:
    msg = register[0]
    target = register[1]

    with my_qbob.if_statement(message == True):
        my_qbob += X(msg)
    
    my_qbob += H(msg)
    my_qbob += teleport(msg, target)    
    my_qbob.returns(Measure([Pauli.PauliX], [target]))

print(my_qbob.build())

operation TeleportMessage (message : Bool) : Result {
    using (register = Qubit[2]) {

        if (message == true) {
            X(register[0]);
        }
        H(register[0]);
        Teleport(register[0], register[1]);
        return Measure([PauliX], [register[1]]);
    }
}


In [15]:
teleport_message = my_qbob.compile()

In [16]:
teleport_message(message=True)

1

In [17]:
teleport_message(message=False)

0

## Prepare entangled state

In [18]:
from typing import List
from qbob.types import Qubit
from qbob.intrinsics import H, CNOT, M
prepare_qbob = qbob.OperationBuilder("PrepareEntangledState", adj=True)
qubits = prepare_qbob.input("qubits", List[Qubit])

prepare_qbob += H(qubits[0])
prepare_qbob += CNOT(qubits[0], qubits[1])

print(prepare_qbob.build())

operation PrepareEntangledState (qubits : Qubit[]) : Unit is Adj {
    H(qubits[0]);
    CNOT(qubits[0], qubits[1]);
}


In [19]:
prepare_qbob.compile()

<Q# callable PrepareEntangledState>

In [20]:
measure_qbob = qbob.OperationBuilder("MeasureEntangledState", debug=True)

with measure_qbob.allocate_qubits("qubits", 2) as q:
    measure_qbob.log_state(q)
    measure_qbob += prepare_qbob(q)
    measure_qbob.log_state(q)
    measure_qbob.returns([M(q[0]), M(q[1])])
    
print(measure_qbob.build())

operation MeasureEntangledState () : Result[] {
    using (qubits = Qubit[2]) {
        // # wave function for qubits with ids (least to most significant): 0;1
        // ∣0❭:	 1.000000 +  0.000000 i	 == 	******************** [ 1.000000 ]     --- [  0.00000 rad ]
        // ∣1❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
        // ∣2❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
        // ∣3❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
        PrepareEntangledState(qubits);
        // # wave function for qubits with ids (least to most significant): 0;1
        // ∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
        // ∣1❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
        // ∣2❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
        // ∣3❭:	 0.7071

In [21]:
measure_op = measure_qbob.compile()

In [22]:
measure_op.simulate()

[0, 0]

## Debugging

In [23]:
my_qbob = qbob.OperationBuilder("TestDumpMachine", debug=True)

with my_qbob.allocate_qubits("q", 3) as q:
    my_qbob += H(q[0])
    my_qbob.log_state(q[0])
    my_qbob += H(q[1])
    my_qbob.log_state(q[1])
    my_qbob += X(q[2])
    my_qbob.log_state(q[2])
    my_qbob.returns([M(q[0]), M(q[1]), M(q[2])])

In [24]:
print(my_qbob.build()) # code contains the state at each line

operation TestDumpMachine () : Result[] {
    using (q = Qubit[3]) {
        H(q[0]);
        // # wave function for qubits with ids (least to most significant): 0
        // ∣0❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
        // ∣1❭:	 0.707107 +  0.000000 i	 == 	***********          [ 0.500000 ]     --- [  0.00000 rad ]
        H(q[1]);
        // # wave function for qubits with ids (least to most significant): 1
        // ∣0❭:	 0.707107 +  0.000000 i	 == 	**********           [ 0.500000 ]     --- [  0.00000 rad ]
        // ∣1❭:	 0.707107 +  0.000000 i	 == 	**********           [ 0.500000 ]     --- [  0.00000 rad ]
        X(q[2]);
        // # wave function for qubits with ids (least to most significant): 2
        // ∣0❭:	 0.000000 +  0.000000 i	 == 	                     [ 0.000000 ]                   
        // ∣1❭:	 1.000000 +  0.000000 i	 == 	******************** [ 1.000000 ]     --- [  0.00000 rad ]
        return [M(q[0]), M(q

In [25]:
print(my_qbob.formatted())

operation TestDumpMachine () : Result[] {
    using (q = Qubit[3]) {
        H(q[0]);
        H(q[1]);
        X(q[2]);
        return [M(q[0]), M(q[1]), M(q[2])];
    }
}


In [26]:
test = my_qbob.compile()
test()

[1, 1, 1]

In [27]:
results = [test() for _ in range(1000)]

In [28]:
import numpy as np

q0, q1, q2 = np.array(results).T

In [29]:
def frequency(result):
    return len(result[result==0])/len(result)

frequency(q0), frequency(q1), frequency(q2)

(0.542, 0.494, 0.0)